# LLM Pentest Execution Workflow Using REST API

This notebook serves as a regression test and tutorial for conducting penetration tests using our API. We'll walk through the entire process, from setting up the test to interpreting results and implementing security measures.

## Table of Contents

1. Getting Started
Selecting an existing endpoint
Creating a new endpoint
2. Understanding and Working with Templates
What is a template?
Selecting an existing template
Creating a new template
3. Initiating a Penetration Test
Starting a one-time test and polling for results
Setting up a scheduled test and polling results
4. Generating and Downloading Reports
Let's begin!

## 1. Connecting to the API
Before making API calls, need to set up your API connection. Our API uses the Bearer authentication scheme with a JWT token. Follow these steps:

1. Create an API key in the user interface in the Admin Console, ensuring it has the necessary roles for penetration testing.
2. Use your API key to obtain a JWT token by calling the v1/auth/issue-jwt-token API endpoint.
3. Use the obtained JWT token for all subsequent API requests.
Here's how to get your JWT token:

In [ ]:
import os
import requests

# Set up API URL and API key
API_URL = os.environ.get(
    "API_URL"
)  # Replace with the actual base URL of your AllTrue tenant
API_KEY = os.environ.get("API_KEY")  # Set your API key as an environment variable
CUSTOMER_ID = os.environ.get("CUSTOMER_ID")  # Replace with your actual customer ID


def get_jwt_token(api_key):
    endpoint = f"{API_URL}/v1/auth/issue-jwt-token"
    headers = {"X-API-Key": f"{api_key}"}
    response = requests.post(endpoint, headers=headers)
    response.raise_for_status()
    return response.json()["access_token"]


# Get the JWT token
JWT_TOKEN = get_jwt_token(API_KEY)
print("JWT token obtained successfully.")


# Function to make API requests
def make_api_request(endpoint, token: str, method="GET", data=None):
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    url = f"{API_URL}{endpoint}"

    response = requests.request(method, url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()

## Testing Your Connection

Now that you have obtained your JWT token, you can test your connection by making a request to the /auth/verify-connection endpoint. This will confirm that your token is valid and that you can successfully communicate with the API.

The /verify-connection endpoint is a simple endpoint that returns a success message if the connection is valid. If successful, you should see a message confirming the connection and the expiration time of your token.

In [ ]:
import json


def test_connection():
    try:
        response = make_api_request("/verify-connection", token=JWT_TOKEN)
        print("Connection verified successfully:")
        print(json.dumps(response, indent=2))
    except requests.exceptions.RequestException as e:
        print(f"Error verifying connection: {e}")


test_connection()

## Starting a Penetration Test

The process of initiating a penetration test using our API involves several key steps:

**1. Select or Create an LLM Endpoint:**

Choose an existing LLM endpoint to test, or
Create a new endpoint for testing

**2. Create a Test Template:**

A test template is a predefined set of test categories to run
Templates are reusable across different endpoints, saving time and ensuring consistency
You can customize templates to focus on specific security aspects or to meet particular compliance requirements

**3. Initiate the Penetration Test:**

Run a one-time test, or
Set up a scheduled test for regular security checks

Let's dive into each of these steps in detail.

## 2. Selecting or Creating an LLM Endpoint

Before we can start a penetration test, we need to identify the target - in this case, an LLM endpoint. You have two options:

1. Select an existing endpoint from your account
2. Create a new endpoint specifically for testing

Let's explore both options:

### Creating a New LLM Endpoint

If you don't have an existing endpoint or want to create a new one specifically for testing, you can use the API to create a new LLM endpoint. To create a llm endpoint, you can use the /v1/inventory/customer/{customer_id}/resources/llm-endpoint endpoint.

The body of the request should include the following parameters:

- resource_type: The type of llm endpoint, detailed below. For example: OpenAIEndpoint.
- display_name: (optional) A human-readable name for the endpoint.
- project_ids: (optional) A list of project IDs to put the endpoint resource in.
- resource_data: A dictionary of key-value pairs that represent the configuration of the endpoint. The keys and values depend on the resource_type.

Below are examples of how to add resources of different types:

#### Adding an OpenAI Compatible Endpoint

OpenAI Compatible Endpoints are endpoints that follow the same signature as OpenAI's v1/chat/completions endpoints. Use this resource type if you want to register and pentest an OpenAI endpoint, or an endpoint that has the same input/output structure.

When registering an OpenAI endpoint, you must provide either an API key or an endpoint-identifier which are used to uniquely identify the resource. If you provide an API key, AllTrue will use the API key to make requests when running a pentest by default. If you would like to use a different API key for pentest, you can override this when initiating a pentest or by setting the pentest_connection_details field in the resource data. Note that you must set the api key in the resource or pentest_connection_details in order to execute pentests.

Here's an example of the resource_data for an OpenAI endpoint with all optional fields set:

In [ ]:
{
    "provider": "SomeCompany",  // Optional. Defaults to "OpenAI" 
    "base_url": "https://api.yourprovider" , // Optional. Defaults to OpenAI's base URL
    "api_key": "your-api-key-here",  // Optional. The API key to use for making requests
    "endpoint_identifier": "your-endpoint-identifier",  // Optional. The endpoint identifier to use for making requests
    "pentest_connection_details": {
      "pentest_url": "https://api.yourprovider.com/v1", // Optional. The URL to use for making requests during a pentest
       "pentest_api_key": "your-pentest-api-key", // Optional. The API key to use for making requests during a pentest
      "model": "gpt-4o"  // Optional. The model to use for making requests during a pentest. Defaults to "gpt-3.5-turbo"
  }
}

Here is an example of how to create a new OpenAI compatible endpoint accepting all the defaults - i.e using the same API key for pentest as for registration and calling OpenAI's base URL.

In [ ]:
OPENAI_API_KEY = os.environ.get(
    "OPENAI_API_KEY"
)  # Replace with your actual OpenAI API key


def create_llm_endpoint(customer_id, token, resource_data):
    endpoint = f"/v1/inventory/customer/{customer_id}/resources/llm-endpoint"
    data = {
        "resource_type": "OpenAIEndpoint",
        "display_name": "My OpenAI Endpoint",
        "resource_data": resource_data,
    }
    response = make_api_request(endpoint, token, method="POST", data=data)
    return response


# Create a new OpenAI endpoint
openai_endpoint_data = {
    "api_key": OPENAI_API_KEY,
    "endpoint_identifier": "openai-endpoint-for-pentest-tutorial",
}

new_openai_endpoint = create_llm_endpoint(
    customer_id=CUSTOMER_ID, token=JWT_TOKEN, resource_data=openai_endpoint_data
)
new_openai_endpoint

### Adding a Custom REST API Endpoint

Sometimes you need to fully specify a custom endpoint that doesn't conform to any standard API structure. This is where custom REST API endpoints come in handy. With custom endpoints, you have complete control over specifying the request and response formats, allowing you to integrate with any LLM API, regardless of its structure.

When creating a custom REST API endpoint, you must fully specify how the TRiSM Hub should call your API during a pentest in the pentest_connection_details field. This includes providing the URL, headers, request body, and expected response format.

- resource_type: The type of llm endpoint, detailed below. For example: CustomLlmEndpoint.
- display_name: (optional) A human-readable name for the endpoint.
- project_ids: (optional) A list of project IDs to put the endpoint resource in.
- pentest_connection_details:
    - pentest_url: The full URL of your API endpoint.
    - headers: A dictionary of headers to send with the request, including any necessary authentication.
    - body: The request body, which must include the placeholder <<PROMPT>> where the actual prompt will be inserted.
    - method: The HTTP method to use, either "POST" or "PUT".
    - response_type: The expected response format, either "json", "text", or "ndjson".
        - json: The response is expected to be in JSON format.
        - text: The response is expected to be plain text.
        - ndjson: The response is expected to be newline-delimited JSON.
    - response_jsonpaths: A list of JSONPath expressions to extract the response from the API output. All matches of these expressions will be concatenated together to form the response.
    - response_error_values: (Optional) A list of values which when found in the result will be considered an error. This is useful if your API returns a successful code even when there is an error.
    - customer_scripts: (Optional) A list of scripts that should be executed for particular endpoint. These scripts are **customer specific** - meaning that an AllTrue engineer will need to write them for you. These should be used only when your API cannot be called directly with a fixed set of headers. Please reach out to AllTrue if you think you need a custom implementation and we will be happy to help.

As an example, here is how we would add an OpenAI Endpoint using this resource type:

In [ ]:
OPENAI_API_KEY = os.environ.get(
    "OPENAI_API_KEY"
)  # Replace with your actual OpenAI API key


def create_custom_api_endpoint(customer_id, token):
    endpoint = f"/v1/inventory/customer/{customer_id}/resources/llm-endpoint"
    data = {
        "resource_type": "CustomLlmEndpoint",
        "display_name": "My Custom API Endpoint",
        "resource_data": {
            "api_key": OPENAI_API_KEY,
            "endpoint_identifier": "custom-endpoint-for-pentest-tutorial",
            "pentest_connection_details": {
                "pentest_url": "https://api.openai.com/v1/chat/completions",
                "headers": {"Authorization": f"Bearer {OPENAI_API_KEY}"},
                "body": {
                    "model": "gpt-4o",
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": "<<PROMPT>>"},
                    ],
                },
                "method": "POST",
                "response_type": "json",
                "response_jsonpaths": ["$.choices[0].text"],
            },
        },
    }
    response = make_api_request(endpoint, token, method="POST", data=data)
    return response


# Create a new custom API endpoint
new_custom_api_endpoint = create_custom_api_endpoint(
    customer_id=CUSTOMER_ID, token=JWT_TOKEN
)
new_custom_api_endpoint

### Selecting an Existing LLM Endpoint

To select an existing LLM endpoint, we first need to retrieve a list of all available endpoints in your account. We can do this using the /v1/inventory/customer/{customer_id}/resources API endpoint.

Here's how to list your existing LLM endpoints:

In [ ]:
def list_endpoints(customer_id, token):
    endpoint = (
        f"/v1/inventory/customer/{customer_id}/resources?resource_category=llm_endpoint"
    )
    response = make_api_request(endpoint, token)
    return response


endpoints = list_endpoints(customer_id=CUSTOMER_ID, token=JWT_TOKEN)
next(
    endpoint
    for endpoint in endpoints["resources"]
    if endpoint["resource_display_name"]
    == "Custom LLM Endpoint (custom-endpoint-for-pentest-tutorial)"
)

## 3. Understanding and Working with Templates

Templates are predefined sets of test categories and configurations used to conduct penetration tests. They allow you to standardize your testing process, ensure consistency across multiple endpoints, and save time by reusing common test configurations.

### What is a template?

A Pentest template is a set of test categories to run on an endpoint. Each category includes a set of test prompts testing for a specific type of security vulnerability.

Templates are created and given names, and are then used to run tests on endpoints. When you run a test, you can specify which template to use. This allows you to reuse the same set of test categories across multiple endpoints, ensuring consistency in your testing process.

### Creating a new template

To create a new template, you can use the POST /v1/llm-pentest/scan_templates endpoint. The body of the request should include the following parameters:

- name: A human-readable name for the template.
- pentest_category_ids: The list of test categories to include in the template. Each category is identified by its ID.
- customer_id: Your customer ID.

Note that either pentest_category_ids or pentest_categories should be provided.

To see the list of available test categories, you can use the /v1/llm-pentest/categories endpoint. For example:


In [ ]:
def list_templates(token):
    endpoint = f"/v1/llm-pentest/categories"
    response = make_api_request(endpoint, token)
    return response


# List available templates
templates = list_templates(token=JWT_TOKEN)
# show the first two category
print(json.dumps(templates["llm_pentest_categories"][:2], indent=2))

Once you have the list of categories you want to include in your template, you can create a new template using the POST /v1/inventory/customer/{customer_id}/resources/pentest-template endpoint. Here's an example of how to create a new template:

In [ ]:
def create_template(customer_id, token, template_data):
    endpoint = f"/v1/llm-pentest/scan_templates"
    response = make_api_request(endpoint, token, method="POST", data=template_data)
    return response


# Example template data
new_template_data = {
    "name": "Template for LLM Pentest Tutorial",
    "pentest_category_ids": [
        "input_denial_of_wallet_attack",
        "renellm",
    ],
    "customer_id": CUSTOMER_ID,
}


# Create a new template
new_template = create_template(
    customer_id=CUSTOMER_ID, token=JWT_TOKEN, template_data=new_template_data
)
new_template

### Selecting an Existing Template

To select an existing template, you can use the /v1/llm-pentest/scan_templates endpoint to retrieve a list of all available templates in your account. Here's how to list your existing templates:

In [ ]:
def list_templates(customer_id, token):
    endpoint = f"/v1/llm-pentest/scan_templates?customer_id={customer_id}"
    response = make_api_request(endpoint, token)
    return response


# List existing templates
templates = list_templates(customer_id=CUSTOMER_ID, token=JWT_TOKEN)

print(
    json.dumps(
        next(
            template
            for template in templates["llm_pentest_scan_templates"]
            if template["name"] == "Template for LLM Pentest Tutorial"
        ),
        indent=2,
    )
)

## 4. Initiating a Penetration Test

Now that we have created an LLM endpoint or selected an existing one, have a template ready, we can initiate a penetration test.

There are two ways to initiate a penetration test:

1. **One-time Test:** Run a single penetration test on the endpoint.
2. **Scheduled Test:** Set up a recurring schedule for running penetration tests on the endpoint.

### Starting a One-time Test

To start a one-time penetration test, you can use the /v1/llm-pentest/scan_templates/{scan_template_id}/start endpoint. The body of the request should include the following parameters:

- resource_instance_id: The ID of the LLM endpoint you want to test.
- customer_id: Your customer ID.
- description: (Optional) A description of the test.
- pentest_connection_details: (Optional) Connection details for the pentest. If not provided, the connection details defined when registering the endpoint will be used.

Pentests can take some time to run. After starting a test, you will receive a task_id that you can use to poll for the status, and an execution_id you can use to download the response.

In [ ]:
def start_one_time_test(scan_template_id, token, test_data):
    endpoint = f"/v1/llm-pentest/scan_templates/{scan_template_id}/start"
    response = make_api_request(endpoint, token, method="POST", data=test_data)
    return response


# Example test data
one_time_test_data = {
    "resource_instance_id": new_openai_endpoint["added_resources"][0][
        "resource_instance_id"
    ],
    "customer_id": CUSTOMER_ID,
    "description": "One-time test for LLM Pentest Tutorial",
}

# Start a one-time test
one_time_test = start_one_time_test(
    scan_template_id=new_template["id"], token=JWT_TOKEN, test_data=one_time_test_data
)
one_time_test

You can then get the status of an execution with /customers/{customer_id}/executions/{execution_id}. This will return the status of the execution, including whether it is running, completed, or errored.

In [ ]:
def get_execution_status(customer_id, execution_id, token):
    endpoint = f"/v1/llm-pentest/customers/{customer_id}/executions/{execution_id}"
    response = make_api_request(endpoint, token)
    return response


# Get the status of the one-time test execution
one_time_test_status = get_execution_status(
    CUSTOMER_ID, one_time_test["llm_pentest_scan_execution_id"], JWT_TOKEN
)
print("Status:", one_time_test_status["status"])

### Setting up a Scheduled Test

To set up a recurring schedule for penetration tests, we need to follow a two-step process:

1. **Define a Job:** This includes specifying the resource (LLM endpoint) to scan and the template to use.
2. **Add a Schedule:** This involves setting the start date, end date, and frequency for the recurring tests.

Let's go through each step:

#### Step 1: Defining a Job

To define a job, we use the /v1/llm-pentest/jobs endpoint. The request body should include:

- resource_instance_id: The ID of the LLM endpoint to test.
- scan_template_id: The ID of the template to use for the tests.
- customer_id: Your customer ID.
- description: (Optional) A description of the job.

#### Step 2: Adding a Schedule

After creating a job, we can add a schedule to it using the /v1/llm-pentest/jobs/{job_id}/schedules endpoint. The request body should include:

- start_date: The date and time to start the recurring tests.
- end_date: (Optional) The date and time to end the recurring tests.
- cron_expression: The cron expression for the schedule. This expression must follow the syntax supported by Amazon EventBridge (https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-scheduled-rule-pattern.html).

Let's implement these steps:

In [ ]:
from typing import Dict, Any, Optional


def create_job(
    token: str,
    resource_instance_id: str,
    scan_template_id: str,
    customer_id: str,
    description: Optional[str] = None,
) -> Dict[str, Any]:
    endpoint = f"/v1/scheduling-jobs/{customer_id}/jobs"
    job_data = {
        "job_type": "LLM_PENTEST_SCAN",
        "enabled": True,
        "data": {
            "resource_instance_id": resource_instance_id,
            "scan_template_id": scan_template_id,
            "description": description,
        },
    }
    response = make_api_request(endpoint, token, method="POST", data=job_data)
    return response

In [ ]:
# Create a job for the endpoint
job = create_job(
    JWT_TOKEN,
    resource_instance_id=new_custom_api_endpoint["added_resources"][0][
        "resource_instance_id"
    ],
    scan_template_id=new_template["id"],
    customer_id=CUSTOMER_ID,
    description="Scheduled test for LLM Pentest Tutorial",
)
print(json.dumps(job, indent=2))

Now that we have created a job, we can add a schedule to it. Let's add a daily schedule for the job.

In [ ]:
def add_schedule(
    token: str,
    customer_id: str,
    job_id: str,
    cron_expression: str,
    start_at: str,
    end_at: Optional[str] = None,
) -> Dict[str, Any]:
    endpoint = f"/v1/scheduling-jobs/{customer_id}/jobs/{job_id}/schedules"
    schedule_data = {
        "customer_id": customer_id,
        "cron_expression": cron_expression,
        "start_at": start_at,
        "end_at": end_at,
    }
    response = make_api_request(endpoint, token, method="POST", data=schedule_data)
    return response

In [ ]:
from datetime import datetime, timedelta

# Add a daily schedule for the job. Will start immediately, and end in a week.
schedule = add_schedule(
    JWT_TOKEN,
    customer_id=CUSTOMER_ID,
    job_id=job["job_id"],
    cron_expression="0 12 * * ? *",  # daily at 12 UTC
    start_at=datetime.now().isoformat(),
    end_at=(datetime.now() + timedelta(days=7)).isoformat(),
)

print(json.dumps(schedule, indent=2))

## 5. Generating and Downloading Reports

Once a penetration test execution is completed, you can generate and download a report of the results. This process involves two steps:

1. Generating the report
2. Downloading the generated report

Let's go through each of these steps in detail.

In [ ]:
# Extract status and execution ID from one-time scan
execution_id = one_time_test["llm_pentest_scan_execution_id"]
status = one_time_test_status["status"]

print(f"Execution ID: {execution_id}")
print(f"Status: {status}")

### Generating a Report

To generate a report for a completed penetration test execution, you can use the /v1/llm-pentest/customers/{customer_id}/executions/generate-report endpoint. This will generate a report for the specified execution ID, and return a pre-signed URL to download the report. This URL remains valid for one hour by default.

In [ ]:
def generate_report(customer_id, execution_id, token):
    endpoint = f"/v1/llm-pentest/customers/{customer_id}/executions/generate-report"
    response = make_api_request(
        endpoint,
        token,
        method="POST",
        data={"llm_pentest_scan_execution_ids": [execution_id]},
    )
    return response

In [ ]:
# Generate a report for the one-time test execution
report = generate_report(CUSTOMER_ID, execution_id, JWT_TOKEN)
report_url = report["report_url"]
print("Download link:", report_url[:100], "...")

### Downloading the Report

Now that we have generated the report and obtained the download URL, we can proceed to download the report. The URL is pre-signed and typically valid for one hour.

To download the report, you can use a simple HTTP GET request to the provided URL. Here's an example of how to download the report using Python's requests library:

In [ ]:
def download_report(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.content


# Download the report
report_content = download_report(report_url)

# Save the report to a file
report_filename = "llm_pentest_report.pdf"
with open(report_filename, "wb") as file:
    file.write(report_content)

print("Report downloaded successfully. Saved as:", report_filename)

### Conclusion

In this comprehensive guide, we've walked through the entire process of conducting penetration tests using our API. We covered:

1. Setting up the API connection
2. Creating and selecting LLM endpoints
3. Understanding and working with templates
4. Initiating both one-time and scheduled penetration tests
5. Generating and downloading reports

By following these steps, you can effectively use our API to conduct thorough security assessments of your LLM endpoints. Regular penetration testing is crucial for maintaining the security and integrity of your AI systems.